In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.read_csv('processed_conversation.csv')

In [ ]:
print("Shape : {}".format(data.shape))
data.head()

In [ ]:
# extract message field and convert message field into list
messages = list(data['lemma'])

In [ ]:
temp = data['message'][84]

In [ ]:
temp

In [ ]:
temp = temp.split('\n')

In [ ]:
for i in range(len(temp)):
    text = re.sub('[^a-zA-Z0-9(+*) \n\.]', ' ', temp[i])
    # text = tp[i]
    print("{} -> {}".format(i, text))

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
sent = [row.split() for row in messages]

In [ ]:
phrases = Phrases(sent, min_count=1, progress_per=10000)

In [ ]:
sentences = phrases[sent]

In [ ]:
from collections import defaultdict

In [ ]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

In [ ]:
from gensim.models import Word2Vec

In [ ]:
w2v_model = Word2Vec(min_count=1, window=2, vector_size=100, alpha=0.03, negative=20, workers=3)

In [ ]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
vocab_len = len(w2v_model.wv)  

In [ ]:
w2v_model.wv.get_vector('welcome')

In [ ]:
w2v_model.wv.key_to_index["hi"]

In [ ]:
# GloVe word embedding technique
import os
from scipy import spatial
import numpy as np

In [ ]:
%%time
embeddings_index = {}
with open('glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    embeddings_index[word] = vector

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
def find_similar_word(emmbedes):
  nearest = sorted(embeddings_index.keys(), key=lambda word: spatial.distance.euclidean(embeddings_index[word], emmbedes))
  return nearest

In [ ]:
find_similar_word(embeddings_index['happy'])[1:10]

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing import text

In [ ]:
tokens = text.Tokenizer()

In [ ]:
tokens.fit_on_texts(messages)

In [ ]:
vocab_size = len(tokens.word_index)

In [ ]:
vocab_size

In [ ]:
print(tokens.index_word)

In [ ]:
encoded_text = tokens.texts_to_sequences(messages)

In [ ]:
encoded_text[0]

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len = 120
x = pad_sequences(encoded_text, maxlen=max_len, padding='post')

In [ ]:
word_vector_matrix = np.zeros((vocab_size+1, 200))

for word, index in tokens.word_index.items():
    vector = embeddings_index.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
    else:
        print(word)